#Clean for GPT-2




In [1]:
import pandas as pd
import csv
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import spacy

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Read Document

In [2]:

with open("/content/drive/MyDrive/Masterthesis/Data/data_unprocessed/combined.tsv", 'r', newline='', encoding='utf-8') as f:
    dialect = csv.Sniffer().sniff(f.read(2048))
    f.seek(0)
    reader = csv.reader(f, dialect)
    header = next(reader)
    combined = pd.DataFrame(reader,columns=["sentiment","text"])


nltk.download('punkt')
nltk.download('stopwords')

#stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:

def sentiment(label):
    if label =="positive":
        return 1
    elif label =="negative":
        return 0


def preprocess_text(text):
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    text = re.sub(email_regex, '', text)

    # lowercase
 #   text = text.lower()

    # replace url addresses with an empty string
    url_pattern = r'(http[s]?:\/\/(?:www\.)?\S+)|(www\.\S+)'
    text = re.sub(url_pattern, '', text)

    # delete brackets and its content
    contains_brackets = r'\([^)]*\)'
    text = re.sub(contains_brackets, '', text)

    # replace phone numbers
    pattern_tel1 = r'\+49\s*\d{1,4}\s*\d{1,4}[\s-]*\d{1,4}'
    pattern_tel2 = r'\+49\s*\(\d\)\s*\d{4}-\d{2}\s*\d'
    pattern_tel3 = r'\+\d{2}\.\d{3}\.\d{3}\.\d{1,4}'

    # replace website addresses with an empty string
    text = re.sub(pattern_tel1, '', text)
    text = re.sub(pattern_tel2, '', text)
    text = re.sub(pattern_tel3, '', text)

    text = re.sub("ue", 'ü', text)
    text = re.sub("ae", 'ä', text)
    text = re.sub("oe", 'ö', text)
    text = re.sub("Ue", 'Ü', text)
    text = re.sub("Ae", 'Ä', text)
    text = re.sub("Oe", 'Ö', text)

    pattern_beginning = r'^[#.%,&+!x]+'
    text = re.sub(pattern_beginning, '', text)

    # from https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b#gistcomment-3315605
    emoji_pattern = re.compile(pattern="["
                    u"\U0001F600-\U0001F64F"  # emoticons
                    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                    u"\U0001F680-\U0001F6FF"  # transport & map symbols
                    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                    u"\U00002500-\U00002BEF"  # chinese char
                    u"\U00002702-\U000027B0"
                    u"\U00002702-\U000027B0"
                    u"\U000024C2-\U0001F251"
                    u"\U0001f926-\U0001f937"
                    u"\U00010000-\U0010ffff"
                    u"\u2640-\u2642"
                    u"\u2600-\u2B55"
                    u"\u200d"
                    u"\u23cf"
                    u"\u23e9"
                    u"\u231a"
                    u"\ufe0f"  # dingbats
                    u"\u3030"
                "]+", flags = re.UNICODE)

    text = re.sub(emoji_pattern, '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'"{2,}', '"', text)
    text = text.replace(' .',".")
    text = text.replace(' ,',", ")
    text = text.replace(' !',"!")
    text = text.replace(' ?',"?")
    text = text.replace(' :',":")
    text = text.replace(' ;',";")
    # text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    text = re.sub(r'@\S', '', text)
    text= re.sub(r'#\S+', ' ', text)
    # text = re.sub('[%s]' % re.escape(punctuation), ' ', text)
    # remove single letters and numbers surrounded by space
    text = re.sub(r'\s[a-z]\s|\s[0-9]\s', ' ', text)

    text = text[0].upper() + text[1:] # uppercase first letter
    text = re.sub(r',(?=[^\s])', ', ', text) #change , without empty spaces afterwards
    text = re.sub(r'\s+', ' ', text)

    return text#" ".join(corrected_words) #text # " ".join(words)



In [4]:
combined1=combined[combined['text'].str.split().apply(len)>10 ]
combined1['preprocessed_text'] = combined1['text'].apply(preprocess_text)
combined1['sentiment'] = combined1['sentiment'].apply(lambda x: sentiment(x))

<ipython-input-4-c0401a8bd39b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined1['preprocessed_text'] = combined1['text'].apply(preprocess_text)
<ipython-input-4-c0401a8bd39b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined1['sentiment'] = combined1['sentiment'].apply(lambda x: sentiment(x))


In [ ]:
#### for SFT
combined_sft=combined1.sample(500000,random_state=345,replace=True)
output_path = "/content/drive/MyDrive/Masterthesis/Data/data_processed/preprocessed500000_for_sft1.tsv"
combined_sft.to_csv(output_path, sep='\t', index=False)